In [1]:
!pip3 install -q pandas numpy matplotlib requests bs4 lxml

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import calendar

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [5]:
months = list(calendar.month_name)[1:]

def getId(tag) :
    return tag['href'].split('/')[-1].split('.html')[0]

In [7]:
url = 'https://www.basketball-reference.com/leagues/NBA_2021_games.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [ ]:
month_urls = [x['href'] for x in soup.find_all('a', href = True) if 'games' in x['href'] 
                and any(m.lower() in x['href'] for m in months)]

In [41]:
def get_monthly_games(month_url) :

    url = f'https://www.basketball-reference.com/{month_url}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    table = soup.find('table')
    games = pd.read_html(str(table))[0].rename(columns = {'Start (ET)': 'Time'})
    games['Date'] = pd.to_datetime(games['Date'])
    games['Home'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][0::2]
    games['Away'] = [x['href'].split('/')[2] for x in table.find_all('a', href = True) if 'teams' in x['href']][1::2]
    games['game_id'] = [getId(x) for x in table.find_all('a', href = True) if 'boxscores' in x['href']][1::2]

    return games[['game_id', 'Date', 'Time', 'Home', 'Away']]

In [43]:
games_monthly = get_monthly_games(month_urls[0])

In [48]:
for gameId, home, away in zip(games_monthly['game_id'], games_monthly['Home'], games_monthly['Away'])  :
    pass

In [ ]:
def get_first_basket(gameId) :
    
    url = f'https://www.basketball-reference.com/boxscores/pbp/{gameId}.html'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    away, home = [x['href'].split('/')[2] for x in soup.find_all('a', href = True) if 'teams' in x['href']][1:3]
    table = soup.find('table')
    table.find('tr', class_ = 'thead').decompose()
    pbp = pd.read_html(str(table))[0]
    cols_ = pbp.columns.to_list()
    cols_[1] = away
    cols_[5] = home
    pbp.columns = cols_

    jump_ball_row_exists = 'Jump ball' in rows[1].text


    pbp['pts_scored'] = pbp['Score'].apply(lambda x: np.array(x.split('-')).astype(int).sum()
                        if '-' in x
                        else np.nan)

    n_actions_before_pts = (pbp['pts_scored'] > 0).argmax() + 1

    # Keep rows until first points scored -- excluding jump ball
    pbp = pbp.head(n_actions_before_pts)[1:]

    # Store player involved
    rows = table.find_all('tr')
    pbp['player'] = [getId(row.find_all('a', href = True)[0]) if row.find('a', href = True) else ''
                 for row in rows[2:n_actions_before_pts+1]]
    
    # Check if miss or make or neither
    pbp = pbp.fillna('')
    pbp['home_miss'] = pbp[home].apply(lambda x: 'misses' in x).astype(int)
    pbp['away_miss'] = pbp[away].apply(lambda x: 'misses' in x).astype(int)
    pbp['home_make'] = pbp[home].apply(lambda x: 'makes' in x).astype(int)
    pbp['away_make'] = pbp[away].apply(lambda x: 'makes' in x).astype(int)
    pbp['shot'] = pbp[['home_miss', 'away_miss', 'home_make', 'away_make']].sum(axis = 1)
    pbp = pbp.copy()[pbp['shot'] == 1]


    # Store jump ball information
    if rows[1].find('a', href = True) :
        jb_away, jb_home, jb_poss = [getId(x) for x in rows[1].find_all('a', href = True)]
    else :
        jb_away, jb_home, jb_poss = None, None, None

    # First basket information
    min, sec = np.array(pbp['Time'].values[-1].split(':')).astype(float)
    time_elapsed = 60 * (12 - min - 1) + (60 - sec)
    pts_scored = pbp['pts_scored'].values[-1]
    num_shots = pbp.shape[0]
    home_misses = pbp['home_miss'].sum()
    away_misses = pbp['away_miss'].sum()
    first_basket_tm = home * pbp['home_make'].values[-1] + away * pbp['away_make'].values[-1]
    first_basket = pbp['player'].values[-1]

    return pd.DataFrame(
                [[gameId, home, away, first_basket, first_basket_tm, time_elapsed, num_shots, pts_scored, home_misses, away_misses,
                  jb_home, jb_away, jb_poss]],
                columns = [
                    'game_id',
                    'Home',
                    'Away',
                    'first_basket',
                    'first_basket_tm',
                    'time_elapsed',
                    'num_shots',
                    'pts_scored',
                    'misses_home',
                    'misses_away',
                    'jumpball_home',
                    'jumpball_away',
                    'jumpball_possession'
                ]
            )

In [9]:
get_first_basket('202304080UTA')

ValueError: not enough values to unpack (expected 3, got 2)

In [17]:
gameId = '202303050BOS'

url = f'https://www.basketball-reference.com/boxscores/pbp/{gameId}.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
away, home = [x['href'].split('/')[2] for x in soup.find_all('a', href = True) if 'teams' in x['href']][1:3]
table = soup.find('table')
table.find('tr', class_ = 'thead').decompose()
pbp = pd.read_html(str(table))[0]
cols_ = pbp.columns.to_list()
cols_[1] = away
cols_[5] = home
pbp.columns = cols_
pbp['pts_scored'] = pbp['Score'].apply(lambda x: np.array(x.split('-')).astype(int).sum()
                    if '-' in x
                    else np.nan)

n_actions_before_pts = (pbp['pts_scored'] > 0).argmax() + 1

In [ ]:

# Keep rows until first points scored -- excluding jump ball
pbp = pbp.head(n_actions_before_pts)[1:]

# Store player involved
rows = table.find_all('tr')
pbp['player'] = [getId(row.find_all('a', href = True)[0]) if row.find('a', href = True) else ''
                 for row in rows[2:n_actions_before_pts+1]]

# Check if miss or make or neither
pbp = pbp.fillna('')
pbp['home_miss'] = pbp[home].apply(lambda x: 'misses' in x).astype(int)
pbp['away_miss'] = pbp[away].apply(lambda x: 'misses' in x).astype(int)
pbp['home_make'] = pbp[home].apply(lambda x: 'makes' in x).astype(int)
pbp['away_make'] = pbp[away].apply(lambda x: 'makes' in x).astype(int)
pbp['shot'] = pbp[['home_miss', 'away_miss', 'home_make', 'away_make']].sum(axis = 1)
pbp = pbp.copy()[pbp['shot'] == 1]


# Store jump ball information
jb_away, jb_home, poss = [getId(x) for x in rows[1].find_all('a', href = True)]

# First basket information
min, sec = np.array(pbp['Time'].values[-1].split(':')).astype(float)
time_elapsed = 60 * (12 - min - 1) + (60 - sec)
pts_scored = pbp['pts_scored'].values[-1]
num_shots = pbp.shape[0]
home_misses = pbp['home_miss'].sum()
away_misses = pbp['away_miss'].sum()
first_basket_tm = home * pbp['home_make'].values[-1] + away * pbp['away_make'].values[-1]
first_basket = pbp['player'].values[-1]

In [18]:
pbp

,Time,NYK,Unnamed: 2,Score,Unnamed: 4,BOS,pts_scored
0,12:00.0,Jump ball: M. Robinson vs. A. Horford (J. Rand...,Jump ball: M. Robinson vs. A. Horford (J. Rand...,Jump ball: M. Robinson vs. A. Horford (J. Rand...,Jump ball: M. Robinson vs. A. Horford (J. Rand...,Jump ball: M. Robinson vs. A. Horford (J. Rand...,NaN
1,11:52.0,I. Quickley makes 3-pt jump shot from 27 ft,+3,3-0,NaN,NaN,3.0
2,11:38.0,NaN,NaN,3-0,NaN,A. Horford misses 3-pt jump shot from 25 ft,3.0
3,11:35.0,NaN,NaN,3-0,NaN,Offensive rebound by M. Smart,3.0
4,11:29.0,NaN,NaN,3-3,+3,D. White makes 3-pt jump shot from 24 ft (assi...,6.0
...,...,...,...,...,...,...,...
554,0:10.0,NaN,NaN,131-129,NaN,Boston full timeout,260.0
555,0:06.0,NaN,NaN,131-129,NaN,Personal take foul by J. Hart (drawn by J. Tatum),260.0
556,0:00.0,NaN,NaN,131-129,NaN,A. Horford misses 3-pt jump shot from 23 ft,260.0
557,0:00.0,NaN,NaN,131-129,NaN,Offensive rebound by Team,260.0


In [15]:
pbp

,Time,DEN,Unnamed: 2,Score,Unnamed: 4,UTA,pts_scored
0,12:00.0,Loose ball foul by N. Jokić (drawn by U. Azubu...,NaN,0-0,NaN,NaN,0.0
1,12:00.0,NaN,NaN,0-0,NaN,Turnover by O. Agbaji (inbound),0.0
2,11:45.0,Personal foul by K. Dunn (drawn by A. Gordon),NaN,0-0,NaN,NaN,0.0
3,11:38.0,J. Murray misses 3-pt jump shot from 25 ft,NaN,0-0,NaN,NaN,0.0
4,11:35.0,NaN,NaN,0-0,NaN,Defensive rebound by U. Azubuike,0.0
...,...,...,...,...,...,...,...
487,0:04.0,V. Čančar misses 3-pt jump shot from 25 ft,NaN,114-118,NaN,NaN,232.0
488,0:01.0,Offensive rebound by I. Smith,NaN,114-118,NaN,NaN,232.0
489,0:00.0,Jump ball: D. Jones vs. Z. Nnaji (K. Dunn gain...,Jump ball: D. Jones vs. Z. Nnaji (K. Dunn gain...,Jump ball: D. Jones vs. Z. Nnaji (K. Dunn gain...,Jump ball: D. Jones vs. Z. Nnaji (K. Dunn gain...,Jump ball: D. Jones vs. Z. Nnaji (K. Dunn gain...,NaN
490,0:00.0,Turnover by I. Smith (lost ball),NaN,114-118,NaN,NaN,232.0


In [22]:
'Jump ball' in rows[1].text

True

In [20]:
rows = table.find_all('tr')
rows[1]

<tr>
<td>12:00.0</td>
<td class="center" colspan="5">Jump ball: <a href="/players/r/robinmi01.html">M. Robinson</a> vs. <a href="/players/h/horfoal01.html">A. Horford</a> (<a href="/players/r/randlju01.html">J. Randle</a> gains possession)</td>
</tr>

In [19]:
rows[1]

<tr>
<td>12:00.0</td>
<td class="">Loose ball foul by <a href="/players/j/jokicni01.html">N. Jokić</a> (drawn by <a href="/players/a/azubuud01.html">U. Azubuike</a>)</td><td class=""> </td><td class="center">0-0</td><td> </td><td> </td>
</tr>